In [75]:
from dash import Dash, dash_table, dcc, html
from dash.dependencies import Input, Output
import pandas as pd
from flask import Flask
from Librerias_SAP import SAP_GUI, Funtions
import pandas as pd
import re
from collections import OrderedDict
from dash import dash_table as dt
from dash import dcc
from dash import html
from dash.dependencies import Input
from dash.dependencies import Output
import numpy as np
import dash_bootstrap_components as dbc
from datetime import datetime, timedelta

In [76]:
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password

Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][0]
password=Keys["Password"][0]
# Initialize session
session=SAP_GUI.SessionSAP(user,password)

In [77]:
# Disposiciones ZSD110
Name_ZSD110_FIRME="Pendiente_Firme"  # Revisar suele no guardar
Name_ZSD110_MERCADEO="Pendiente_Mercadeo"  # Revisar suele no guardar
Name_ZSD110_TOTAL="Pendiente_Total"  # Revisar suele no guardar


default_column_Canal01_Pendientes=['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Pedido', 'Clase Pedi', 'Creado el', 'Pedido.1', 'Posición', 'Material',
       'Vlr.Neto P', 'Moneda', 'Cantidad P', 'Cantidad C', 'UM venta',
       'Vlr.Neto C', 'Status Glo', 'Status Tot', 'Status Ent', 'Status Ent.1',
       'Status Ent.2']  


default_column_Canal01_Entregado=['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Entrega', 'Posicion E', 'F.Creacion', 'Pedido', 'Clase Pedi',
       'F.Creacion.1', 'Pedido.1', 'Posicion P', 'Material', 'Moneda',
       'Cantidad E', 'UM venta', 'Valor Neto', 'StTotPick.', 'StatGlWM',
       'StTotMovMe', 'Stat.fact.', 'Status PT']


default_column_Pedidos=['OrgVt', 'CDis', 'BqEn', 'Solic.', 'PrimFeEntr', 'ÚltEntrega',
       'FePrefEnt.', 'ClVt', 'Valor neto', 'Mon.', 'Doc.venta', 'ST',
       'Solicitante', 'Nº pedido cliente', 'Descrip.breve', 'Nº de cliente 1',
       'Contador de pedidos', 'Dirección', 'Barrio', 'Población', 'Teléfono']


Columns_Data_ZSD10_Exito_Cencosub_ZVMI=['Fecha doc.', 'Solicitante', 'Cliente', 'Nº de cliente 1', 'Marca',
       'Material', 'Número de material', 'Unidades', 'UM', 'ValorNeto',
       'Nº pedido cliente', 'Doc.venta', 'ClVt', 'Solic.', 'OrgVt', 'CDis',
       'OfVta', 'GVen', 'Organización de ventas', 'Mon.', 'Jquía.productos',
       'Total Pedidos', 'Plataforma', 'Fe.PedCpas', 'Creado por']

Columns_Data_ZSD038_Exito_Cencosub_Rechazados=['Doc.comer.', 'Fecha doc.', 'Destinat.', 'Nombre 1', 'Cód.Vend.',
       'Nombre vendedor', 'MR', 'Denominación', 'Material', 'Denominación.1',
       'Marca', 'Categoría', 'Ctd.confirmada', 'Valor neto', 'Mon.', 'CDis',
       'Solic.', 'Nombre']

"""----------------------------------------------LEER ARCHIVOS--------------------------------"""
#Data Pendientes
Canal01_Pendiente=pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\Canal01_PEN.txt",skiprows=1,delimiter="\t")
Canal01_Pendiente=Funtions.Clean_Columns(Canal01_Pendiente)
Canal01_Pendiente=Funtions.default_column(default_column_Canal01_Pendientes,Canal01_Pendiente)

#Dta Entregados
Canal01_Entregado=pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\Canal01_ENT.txt",skiprows=1,delimiter="\t")
Canal01_Entregado=Funtions.Clean_Columns(Canal01_Entregado)
Canal01_Entregado=Funtions.default_column(default_column_Canal01_Entregado,Canal01_Entregado)


"""""-------------------------------------------Limpio los datos-----------------------------------"""
#Canal01_Pendiente['Vlr.Neto C'], Canal01_Pendiente['Cantidad P']=Funtions.Clean_Num_List(Canal01_Entregado['Vlr.Neto C'],Canal01_Entregado['Cantidad P'])
Canal01_Entregado['Valor Neto'], Canal01_Entregado['Cantidad E']=Funtions.Clean_Num_List(Canal01_Entregado['Valor Neto'],Canal01_Entregado['Cantidad E'])


In [78]:
"""-----------------Datos descriptivos de interes-----------------------------------"""
#Lineas Totales
Lines_Canal01_Pendiente=len(Canal01_Pendiente)
Lines_Canal01_Entregado=len(Canal01_Entregado)

In [79]:
"""------------------------Optener datos de Agenda_Exito y Cencosub------------------"""
#Agenda Exito y Cencosub
Agenda_Exito=["0085","0020","0146","0149","0050","0138","0045"]
Agenda_Cencosub=["93","122","127","95","60"]


"""-----------------------------Aplico filtro en Agenda exito----------------------------------"""
Filtro_Agenda_Exito=Funtions.Datos_Agenda(Canal01_Entregado['Pedido.1'],Agenda_Exito)
Filtro_Agenda_Cencosub=Funtions.Search_agenda(Canal01_Entregado["Pedido.1"],Agenda_Cencosub)


"""------------------------------Canal 01 Entrega con y sin agenda Exito-----------------------"""
Filtro_Canal01_Entregado_Exito=(Canal01_Entregado['Pedido.1'].isin(Filtro_Agenda_Exito))   #Excluyo con ~
Canal01_Entregado_Agenda_Exito=Canal01_Entregado.loc[Filtro_Canal01_Entregado_Exito].reset_index(drop=True)
Canal01_Entregado_Sin_Agenda_Exito=Canal01_Entregado.loc[~Filtro_Canal01_Entregado_Exito].reset_index(drop=True)


"""------------------------------Canal 01 Entrega con y sin agenda Cencosub-----------------------"""
Filtro_Canal01_Entregado_Cencosub=(Canal01_Entregado['Pedido.1'].isin(Filtro_Agenda_Cencosub))   #Excluyo con ~
Canal01_Entregado_Agenda_Cencosub=Canal01_Entregado.loc[Filtro_Canal01_Entregado_Cencosub].reset_index(drop=True)
Canal01_Entregado_Sin_Agenda_Cencosub=Canal01_Entregado.loc[~Filtro_Canal01_Entregado_Cencosub].reset_index(drop=True)


In [80]:
#Creo las tablas dinamicas de cliente Exito y Cencosub

Table_dinamica_Exito_entrega=Canal01_Entregado_Agenda_Exito.pivot_table(index=['Pedido.1','Clase Pedi'],aggfunc={'Material':'count','Cantidad E':sum,'Valor Neto':sum}).reset_index()
Lineas_table_dinamica_Exito_entrega=sum(Table_dinamica_Exito_entrega["Material"])


Table_dinamica_Cencosub_entrega=Canal01_Entregado_Agenda_Cencosub.pivot_table(index=['Pedido.1','Clase Pedi'],aggfunc={'Material':'count','Cantidad E':sum,'Valor Neto':sum}).reset_index()
Lineas_table_dinamica_Cencosub_entrega=sum(Table_dinamica_Cencosub_entrega["Material"])

In [81]:
pedidos_Exito_Cencosub=pd.concat([Table_dinamica_Cencosub_entrega['Pedido.1'],Table_dinamica_Exito_entrega['Pedido.1']])

#Busco en la tansaccion ZSD037 los pedidos de las tablas dinamicas tanto de Exito y Cencosub

Transsaccion_ZSD037="zsd037"
SAP_GUI.Search_Pedidos_ZSD037(Transsaccion_ZSD037,pedidos_Exito_Cencosub,session)

In [82]:
Name_ZSD037="Pedidos pendientes Exito y Cencosub"
Ruta_ZSD037_Exito_Cencosub="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_ZSD037,session,Ruta_ZSD037_Exito_Cencosub)

In [83]:
# Utilizo la transaccion para buscar ZSD037 en entrega para las fechas
url_ZSD037_Pedidos_Exito_Cencosub="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_ZSD037)+".txt"

Data_ZSD037_Pedidos_Exito_Cencosub=pd.read_csv(url_ZSD037_Pedidos_Exito_Cencosub,skiprows=5,delimiter="\t")
Data_ZSD037_Pedidos_Exito_Cencosub=Funtions.Clean_Columns(Data_ZSD037_Pedidos_Exito_Cencosub)
Data_ZSD037_Pedidos_Exito_Cencosub=Funtions.default_column(default_column_Pedidos,Data_ZSD037_Pedidos_Exito_Cencosub)
Data_ZSD037_Pedidos_Exito_Cencosub=Data_ZSD037_Pedidos_Exito_Cencosub[Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente'].notnull()]



In [84]:
Data_ZSD037_Pedidos_Exito_Cencosub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4940 entries, 0 to 4939
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   OrgVt                4940 non-null   float64
 1   CDis                 4940 non-null   float64
 2   BqEn                 0 non-null      float64
 3   Solic.               4940 non-null   float64
 4   PrimFeEntr           4940 non-null   object 
 5   ÚltEntrega           4940 non-null   object 
 6   FePrefEnt.           4940 non-null   object 
 7   ClVt                 4940 non-null   object 
 8   Valor neto           4940 non-null   object 
 9   Mon.                 4940 non-null   object 
 10  Doc.venta            4940 non-null   float64
 11  ST                   4940 non-null   object 
 12  Solicitante          4940 non-null   object 
 13  Nº pedido cliente    4940 non-null   object 
 14  Descrip.breve        4940 non-null   object 
 15  Nº de cliente 1      4417 non-null   o

In [85]:
try:
    Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente']=SAP_GUI.Clean_int_to_str(Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente'])   #OrgVt   #CDis  #4940
except:
    None  #No ese aplica metódo

In [86]:
#Dataframe de interes
Data_ZSD037_Pedidos_Pendientes_Fecha=Data_ZSD037_Pedidos_Exito_Cencosub[['Nº pedido cliente','PrimFeEntr','ÚltEntrega']].drop_duplicates()


In [87]:
#---------------------------------Tabla dinamica Exito---------------------------------------------
Tabla_Agenda_Exito=pd.merge(Data_ZSD037_Pedidos_Pendientes_Fecha,Table_dinamica_Exito_entrega,how="right",left_on="Nº pedido cliente",right_on="Pedido.1")[['Nº pedido cliente', 'PrimFeEntr', 'ÚltEntrega',
       'Clase Pedi', 'Cantidad E', 'Material', 'Valor Neto']]

Agenda_name_Exito={"0085":"FUNZA","0020":"VEGAS","0146":"BARRANQUILLA","0149":"BUCARAMANGA","0050":"CALI","0138":"PEREIRA","0045":"SURTIMAYORISTAS"}

Tabla_Agenda_Exito["Plataforma"]=Tabla_Agenda_Exito["Nº pedido cliente"].apply(lambda x: Agenda_name_Exito[re.findall("("+"|".join(list(Agenda_name_Exito.keys()))+")",x)[0]] if re.findall("("+"|".join(list(Agenda_name_Exito.keys()))+")",x)!=[] else x)


In [88]:
#Dataframe de interes

Tabla_Agenda_Cencusub=pd.merge(Data_ZSD037_Pedidos_Pendientes_Fecha,Table_dinamica_Cencosub_entrega,how="right",left_on="Nº pedido cliente",right_on="Pedido.1")[['Nº pedido cliente', 'PrimFeEntr', 'ÚltEntrega',
       'Clase Pedi', 'Cantidad E', 'Material', 'Valor Neto']]

Agenda_name_Cencosub={"93-":"MEDELLIN","122-":"BARRANQUILLA","127-":"BUCARAMANGA","95-":"CALI","60-":"BOGOTA"}

Tabla_Agenda_Cencusub["Plataforma"]=Tabla_Agenda_Cencusub["Nº pedido cliente"].apply(lambda x: Agenda_name_Cencosub[re.findall("\d*-",x)[0]] if re.findall("\d*-",x)[0] in list(Agenda_name_Cencosub.keys()) else None)


In [89]:
Tabla_Agenda_Cencusub

,Nº pedido cliente,PrimFeEntr,ÚltEntrega,Clase Pedi,Cantidad E,Material,Valor Neto,Plataforma
0,122-439976,08.06.2023,08.06.2023,ZVMI,723.0,334,11598587.0,BARRANQUILLA
1,127-443841,08.06.2023,08.06.2023,ZVMI,1820.0,544,28247168.0,BUCARAMANGA
2,60-1158908,02.06.2023,02.06.2023,ZEDI,118.0,25,5119346.0,BOGOTA
3,60-1158916,02.06.2023,04.06.2023,ZEDI,12.0,1,378348.0,BOGOTA
4,60-1159026,05.06.2023,05.06.2023,ZVMI,6098.0,2524,93697730.0,BOGOTA
5,93-485553,08.06.2023,08.06.2023,ZVMI,1008.0,476,15120226.0,MEDELLIN
6,95-493513,08.06.2023,08.06.2023,ZVMI,1123.0,389,14674390.0,CALI


# Segunda parte bot

In [90]:
# Pendiente Firme
Name_ZSD110_FIRME="Pendiente_Firme"  # Revisar suele no guardar

# Pendiente Mercadeo
Name_ZSD110_MERCADEO="Pendiente_Mercadeo"  # Revisar suele no guardar

# Pendiente Total 
Name_ZSD110_TOTAL="Pendiente_Total"  # Revisar suele no guardar


In [91]:
# ZSD110 FIRME
url_ZSD110_FIRME="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_ZSD110_FIRME)+".txt"
Data_ZSD110_FIRME=pd.read_csv(url_ZSD110_FIRME,skiprows=1,delimiter="\t")
Data_ZSD110_FIRME=Funtions.Clean_Columns(Data_ZSD110_FIRME)
Data_ZSD110_FIRME["Lineas"]=Funtions.Clean_Num_List(Data_ZSD110_FIRME["Lineas"])


# ZSD110 MERCADE0
url_ZSD110_MERCADEO="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_ZSD110_MERCADEO)+".txt"
Data_ZSD110_MERCADEO=pd.read_csv(url_ZSD110_MERCADEO,skiprows=1,delimiter="\t")
Data_ZSD110_MERCADEO=Funtions.Clean_Columns(Data_ZSD110_MERCADEO)
Data_ZSD110_MERCADEO["Lineas"]=Funtions.Clean_Num_List(Data_ZSD110_MERCADEO["Lineas"])


# ZSD110 Pendiente Total
url_ZSD110_PENDIENTES="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_ZSD110_TOTAL)+".txt"
Data_ZSD110_PENDIENTES=pd.read_csv(url_ZSD110_PENDIENTES,skiprows=1,delimiter="\t")
Data_ZSD110_PENDIENTES=Funtions.Clean_Columns(Data_ZSD110_PENDIENTES)
Data_ZSD110_PENDIENTES["Lineas"]=Funtions.Clean_Num_List(Data_ZSD110_PENDIENTES["Lineas"])

"""---------------------CAMBIAR INDICES------------------------------------------"""
Channels_Name={1:"MODERNO",2:"TRADICIONAL",7:"TIENDAS DE BELLEZA",8:"PROFESIONAL",12:"INSTITUCIONAL",14:"HARD DISCOUNTERS",15:"ECOMMERCE",19:"ALTERNATIVO",50:"ALMACENES INTERNOS"}

#Aplicar transformacion
Data_ZSD110_MERCADEO["Can.distr."]=Data_ZSD110_MERCADEO["Can.distr."].replace(Channels_Name)
Data_ZSD110_FIRME["Can.distr."]=Data_ZSD110_FIRME["Can.distr."].replace(Channels_Name)
Data_ZSD110_PENDIENTES["Can.distr."]=Data_ZSD110_PENDIENTES["Can.distr."].replace(Channels_Name)

In [92]:
Data_ZSD110_PENDIENTES

,Can.distr.,Pedidos,Lineas,Unidades,Valor Neto
0,MODERNO,336,913.0,"24,731","388,448,000"
1,TRADICIONAL,54,608.0,"21,122","208,136,200"
2,TIENDAS DE BELLEZA,107,1428.0,"11,730","138,367,500"
3,PROFESIONAL,81,803.0,"2,681","97,702,600"
4,INSTITUCIONAL,2,2.0,370,"4,477,200"
5,HARD DISCOUNTERS,0,0.0,0,0
6,ECOMMERCE,17,77.0,82,"1,517,100"
7,ALTERNATIVO,4,125.0,"26,703","169,478,200"
8,ALMACENES INTERNOS,0,0.0,0,0
9,NaN,601,3956.0,"87,419","1,008,126,800"


In [93]:
#Aplicar consolidado de todas las tablas Mercadeo Firme Total
Consolidado_Mercadeo_Firme_Total=pd.merge(pd.merge(Data_ZSD110_MERCADEO,Data_ZSD110_FIRME,how="right",on="Can.distr."),Data_ZSD110_PENDIENTES,how="right",on="Can.distr.")
Consolidado_Mercadeo_Firme_Total=Consolidado_Mercadeo_Firme_Total.fillna(0)

In [94]:

Column_Consolidado_Mercadeo_Firme_Total=['Can.distr.', 'Pedidos', 'Lineas', 'Unidades', 'Valor Neto',
       'Pedidos', 'Lineas', 'Unidades', 'Valor Neto', 'Pedidos',
       'Lineas', 'Unidades', 'Valor Neto']

Consolidado_Clientes=Funtions.default_column(Column_Consolidado_Mercadeo_Firme_Total,Consolidado_Mercadeo_Firme_Total)

In [95]:
#Creo tabla total de la linea de clientes

Datos_lineas_clientes=pd.DataFrame([[1,"ECOMMERCE",0],[2,"Cencosud",0],[3,"Éxito Agenda",0],
              [4,"Canal 1",0],[5,"Canales 2,7,8",0],[6,"BAZAR",0],[7,"Mercadeo",0],
              [8,"Canales 14, 19 Citas",0],[9,"Extra agenda",0]],columns=["Prioridad","Clientes","Cantidad"])

In [96]:
#Calclulo las celdas calculadas

valor_ecommerce =Consolidado_Clientes.loc[6][6]   #6 Ecommers  y 6 Lineas
valor_canal_2_7_8=Consolidado_Clientes.loc[1][6]+Consolidado_Clientes.loc[2][6]+Consolidado_Clientes.loc[3][6]   #Tradiconal suma(1,2,3)
valor_Bazar=Consolidado_Clientes.loc[4][6]   
valor_Mercadeo=sum([Consolidado_Clientes.loc[i][2] for i in range(0,len(Consolidado_Clientes)-1)])
valor_canal_14_19=Consolidado_Clientes.loc[5][6]+Consolidado_Clientes.loc[7][6]

In [97]:
#Remplazo valores en las celdas calculadas
Datos_lineas_clientes["Cantidad"][0] = valor_ecommerce
Datos_lineas_clientes["Cantidad"][4] = valor_canal_2_7_8
Datos_lineas_clientes["Cantidad"][5] = valor_Bazar
Datos_lineas_clientes["Cantidad"][6] = valor_Mercadeo
Datos_lineas_clientes["Cantidad"][7] = valor_canal_14_19

C:\Users\prac.ingindustrial2\AppData\Local\Temp\ipykernel_21184\602017340.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Datos_lineas_clientes["Cantidad"][0] = valor_ecommerce
C:\Users\prac.ingindustrial2\AppData\Local\Temp\ipykernel_21184\602017340.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Datos_lineas_clientes["Cantidad"][4] = valor_canal_2_7_8
C:\Users\prac.ingindustrial2\AppData\Local\Temp\ipykernel_21184\602017340.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [98]:
Datos_lineas_clientes   #Total lineas del total - lineas cencosub - lineas exito - canal 1

,Prioridad,Clientes,Cantidad
0,1,ECOMMERCE,98
1,2,Cencosud,0
2,3,Éxito Agenda,0
3,4,Canal 1,0
4,5,"Canales 2,7,8",2659
5,6,BAZAR,2
6,7,Mercadeo,177
7,8,"Canales 14, 19 Citas",125
8,9,Extra agenda,0


# Parte 4

In [99]:
transsaccion="ZSD035D"
variant="JESPINOSAP"
provision="INFORME FACTU"

In [100]:

#SAP_GUI.Search_ZSD035D(transsaccion,session,provision,"15.06.2023",variant)

In [101]:
Un_dia_atras=(datetime.now()-timedelta(days=1)).date()
Un_dia_atras=Funtions.stand_day(Un_dia_atras)
SAP_GUI.Search_ZSD035D(transsaccion,session,provision,Un_dia_atras,variant)
# Pendiente Firme
Name_ZSD035D="Pendiente_Clientes"  # Revisar suele no guardar
SAP_GUI.Export_TXT2(Name_ZSD035D,session)

In [102]:
# Pendiente Firme
url_ZSD035D="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\Archivos_CSV\\"+str(Name_ZSD035D)+".txt"
Data_ZSD035D=pd.read_csv(url_ZSD035D,skiprows=5,delimiter="\t")
Data_ZSD035D=Funtions.Clean_Columns(Data_ZSD035D)
Data_ZSD035D=Data_ZSD035D[Data_ZSD035D['Material'].notnull()]
#Data_ZSD035D["Lineas"]=Data_ZSD035D["Lineas"].apply(lambda x:Funtions.Clean_num(x))    
#Tener encuenta esta lineas puede este en dato numerico

In [103]:
#Limpio datos de la columnas a numeros

Data_ZSD035D['Cantidad'], Data_ZSD035D['Valor Neto']= Funtions.Clean_Num_List(Data_ZSD035D['Cantidad'],Data_ZSD035D['Valor Neto'])


In [104]:
#Canal01_Entregado_Exluido.pivot_table(,,['Material', 'Valor Neto'])

Table_Data_ZSD035D=Data_ZSD035D.pivot_table(index=['CDis'],aggfunc={'Material':'count','Pedido':'count','Valor Neto':sum,'Cantidad':sum}).reset_index()[['CDis','Pedido', 'Material', 'Cantidad', 'Valor Neto']]
Table_Data_ZSD035D=Funtions.Clean_Columns(Table_Data_ZSD035D)

Table_Data_ZSD035D

,CDis,Pedido,Material,Cantidad,Valor Neto
0,1.0,552,552,4268.0,104504422.0
1,2.0,498,498,7426.0,87883228.0
2,7.0,1650,1650,16916.0,161902278.0
3,8.0,1042,1042,3926.0,140322092.0
4,12.0,5,5,2891.0,22247251.0
5,15.0,167,167,203.0,3564549.0
6,19.0,3,3,444.0,3409608.0
7,20.0,36,36,422.0,1883042.0


# Aqui silulo los datos sin tener en cuenta el excel

In [105]:
# Debo de filtrar por ZVMI y exito y cencusub del día anterior  

"""Tendria que usar el excel de sharepoint como sugerencia"""

filtro_ZVMI_Exito=Tabla_Agenda_Exito["Clase Pedi"]=="ZVMI"
filtro_ZVMI_Cencosub=Tabla_Agenda_Exito["Clase Pedi"]=="ZVMI"

Tabla_Agenda_Exito_ZVMI=Tabla_Agenda_Exito.loc[filtro_ZVMI_Exito]
Tabla_Agenda_Cencosub_ZVMI=Tabla_Agenda_Cencusub.loc[filtro_ZVMI_Cencosub]

In [106]:
Lineas_Tabla_Agenda_Exito_ZVMI=Tabla_Agenda_Exito_ZVMI["Cantidad E"].sum()
Lineas_Tabla_Agenda_Cencosub_ZVMI=Tabla_Agenda_Cencosub_ZVMI["Cantidad E"].sum()
Unidades_Tabla_Agenda_Exito_ZVMI=Tabla_Agenda_Exito_ZVMI.Material.sum()
Unidades_Tabla_Agenda_Cencosub_ZVMI=Tabla_Agenda_Cencosub_ZVMI.Material.sum()
Valor_neto_Tabla_Agenda_Exito_ZVMI=Tabla_Agenda_Exito_ZVMI["Valor Neto"].sum()
Valor_neto_Tabla_Agenda_Cencosub_ZVMI=Tabla_Agenda_Cencosub_ZVMI["Valor Neto"].sum()

In [107]:
Lineas_Tabla_Agenda_Exito_ZVMI
Valor_neto_Tabla_Agenda_Exito_ZVMI

76711649.0

In [108]:
Tabla_Agenda_Exito_ZVMI

,Nº pedido cliente,PrimFeEntr,ÚltEntrega,Clase Pedi,Cantidad E,Material,Valor Neto,Plataforma
0,0020631946,01.06.2023,01.06.2023,ZVMI,14.0,11,337034.0,VEGAS
4,0050432811,03.06.2023,03.06.2023,ZVMI,2270.0,724,39626805.0,CALI
5,0050432812,03.06.2023,03.06.2023,ZVMI,2.0,2,26218.0,CALI
6,0050432813,03.06.2023,03.06.2023,ZVMI,893.0,437,7897508.0,CALI
7,0050432814,03.06.2023,03.06.2023,ZVMI,14.0,8,438721.0,CALI
8,0050432815,03.06.2023,03.06.2023,ZVMI,486.0,293,3757894.0,CALI
10,0050432823,03.06.2023,03.06.2023,ZVMI,21.0,6,275247.0,CALI
11,0050432824,03.06.2023,03.06.2023,ZVMI,24.0,13,434074.0,CALI
17,0050432853,03.06.2023,03.06.2023,ZVMI,5.0,4,144068.0,CALI
26,0085751310,03.06.2023,03.06.2023,ZVMI,1367.0,783,19838308.0,FUNZA


In [109]:
# Tambien descargar la base de datos central para ver lo de los días

In [110]:
Canal01_Entregado_Agenda_Exito.columns

Index(['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Entrega', 'Posicion E', 'F.Creacion', 'Pedido', 'Clase Pedi',
       'F.Creacion.1', 'Pedido.1', 'Posicion P', 'Material', 'Moneda',
       'Cantidad E', 'UM venta', 'Valor Neto', 'StTotPick.', 'StatGlWM',
       'StTotMovMe', 'Stat.fact.', 'Status PT'],
      dtype='object')

In [111]:
Tabla_Agenda_Exito_ZVMI["Nº pedido cliente"]

0     0020631946
4     0050432811
5     0050432812
6     0050432813
7     0050432814
8     0050432815
10    0050432823
11    0050432824
17    0050432853
26    0085751310
27    0085751313
28    0085751314
29    0085751354
Name: Nº pedido cliente, dtype: object

In [112]:
# ZSD110 FIRME
url_ZSD037_Pedidos_Exito_Cencosub="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_ZSD037)+".txt"

Data_ZSD110_Pedidos_Exito_Cencosub=pd.read_csv(url_ZSD037_Pedidos_Exito_Cencosub,skiprows=5,delimiter="\t")
Data_ZSD110_Pedidos_Exito_Cencosub=Funtions.Clean_Columns(Data_ZSD110_Pedidos_Exito_Cencosub)
Data_ZSD110_Pedidos_Exito_Cencosub=Funtions.default_column(default_column_Pedidos,Data_ZSD110_Pedidos_Exito_Cencosub)
Data_ZSD110_Pedidos_Exito_Cencosub=Data_ZSD110_Pedidos_Exito_Cencosub[Data_ZSD110_Pedidos_Exito_Cencosub['Nº pedido cliente'].notnull()]


In [113]:
Data_ZSD110_Pedidos_Exito_Cencosub

,OrgVt,CDis,BqEn,Solic.,PrimFeEntr,ÚltEntrega,FePrefEnt.,ClVt,Valor neto,Mon.,...,ST,Solicitante,Nº pedido cliente,Descrip.breve,Nº de cliente 1,Contador de pedidos,Dirección,Barrio,Población,Teléfono
0,1000.0,1.0,NaN,1.000000e+09,01.06.2023,01.06.2023,01.06.2023,ZVMI,"31,931",COP,...,C,ALMACENES EXITO SA,0020631946,Tratado completamente,EXITO - ALMACENES EXITO,1,CR 48 32 B SUR 139,NaN,Envigado,3396565.0
1,1000.0,1.0,NaN,1.000000e+09,01.06.2023,01.06.2023,01.06.2023,ZVMI,"31,931",COP,...,C,ALMACENES EXITO SA,0020631946,Tratado completamente,ALMACENES EXITO SA,1,CR 48 32 B SUR 139,NaN,Envigado,3396565.0
2,1000.0,1.0,NaN,1.000000e+09,01.06.2023,01.06.2023,01.06.2023,ZVMI,"31,931",COP,...,C,ALMACENES EXITO SA,0020631946,Tratado completamente,CEDI LAS VEGAS,1,CR 48 32 B SUR 139,NaN,Envigado,3396565.0
3,1000.0,1.0,NaN,1.000000e+09,01.06.2023,01.06.2023,01.06.2023,ZVMI,"31,931",COP,...,C,ALMACENES EXITO SA,0020631946,Tratado completamente,ALMACENES EXITO SA,1,CR 48 32 B SUR 139,NaN,Envigado,3396565.0
4,1000.0,1.0,NaN,1.000000e+09,01.06.2023,01.06.2023,01.06.2023,ZVMI,"31,931",COP,...,C,ALMACENES EXITO SA,0020631946,Tratado completamente,NaN,1,CR 48 32 B SUR 139,NaN,Envigado,3396565.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4935,1000.0,1.0,NaN,1.000007e+09,08.06.2023,08.06.2023,08.06.2023,ZVMI,"1,981,461",COP,...,C,CENCOSUD COLOMBIA SA,95-493513,Tratado completamente,JUMBO CHIPICHAPE,1,CR 35 A 16 80 BG 9 PARQUE INDUSTRIA,NaN,Bogotá,7429800.0
4936,1000.0,1.0,NaN,1.000007e+09,08.06.2023,08.06.2023,08.06.2023,ZVMI,"1,981,461",COP,...,C,CENCOSUD COLOMBIA SA,95-493513,Tratado completamente,RV-FLOR JAZMIN TORRES BUSTAMANTE,1,CR 35 A 16 80 BG 9 PARQUE INDUSTRIA,NaN,Bogotá,7429800.0
4937,1000.0,1.0,NaN,1.000007e+09,08.06.2023,08.06.2023,08.06.2023,ZVMI,"1,981,461",COP,...,C,CENCOSUD COLOMBIA SA,95-493513,Tratado completamente,RV- ALEJANDRO CALAD POSADA,1,CR 35 A 16 80 BG 9 PARQUE INDUSTRIA,NaN,Bogotá,7429800.0
4938,1000.0,1.0,NaN,1.000007e+09,08.06.2023,08.06.2023,08.06.2023,ZVMI,"1,981,461",COP,...,C,CENCOSUD COLOMBIA SA,95-493513,Tratado completamente,CPV-KATHERINE ABAUNZA BONILLA,1,CR 35 A 16 80 BG 9 PARQUE INDUSTRIA,NaN,Bogotá,7429800.0


In [114]:
Tabla_Agenda_Exito_ZVMI["Nº pedido cliente"].to_clipboard(index=False, header=False)

In [115]:
Tabla_Agenda_Exito_Cencosub_ZVMI=pd.concat([Tabla_Agenda_Exito_ZVMI["Nº pedido cliente"],Tabla_Agenda_Cencosub_ZVMI["Nº pedido cliente"]])

In [116]:
transsaccion_ZSD10="ZSD10"
provision_ZSD10="PEDIDOS"
variant_ZSD10="JESPINOSAP"




def Search_ZSD10(transsaccion,Series,variant,provision,session):
    """
    Transsaccion: Transsacion a buscar
    Series: Columna del dataframe que quiero copiar
    session: session del usuario
    variant: Variante a buscar
    provision: disposicion de interes
    session: session activa
    """
    session.StartTransaction(transsaccion)
    session.findById("wnd[0]/tbar[1]/btn[17]").press()
    session.findById("wnd[1]/usr/txtV-LOW").text = provision
    session.findById("wnd[1]/usr/txtENAME-LOW").text = variant
    session.findById("wnd[1]/usr/txtV-LOW").caretPosition = 8
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    Series=Series.to_clipboard(index=False, header=False)
    session.findById("wnd[0]/usr/btn%_SP$00013_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    try:
        tabla=session.findById("wnd[0]/usr/cntlCONTAINER/shellcont/shell")
        return(tabla)
    except:
        return(session)


Search_ZSD10(transsaccion_ZSD10,Tabla_Agenda_Exito_Cencosub_ZVMI,variant_ZSD10,provision_ZSD10,session)
Name_ZSD010_Exito_Cencosub_ZVMI="Exito_Cencosub_ZVMI"
Ruta_Exito_ZSD010_ZVMI="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_ZSD010_Exito_Cencosub_ZVMI,session,Ruta_Exito_ZSD010_ZVMI)


In [117]:
# Exportar de la transsaccion ZSD10 exporta los archivos

url_ZSD10_Exito_Cencosub_ZVMI=Ruta_Exito_ZSD010_ZVMI+Name_ZSD010_Exito_Cencosub_ZVMI+".txt"
Data_ZSD10_Exito_Cencosub_ZVMI=pd.read_csv(url_ZSD10_Exito_Cencosub_ZVMI,skiprows=5,delimiter="\t")
Data_ZSD10_Exito_Cencosub_ZVMI=Funtions.Clean_Columns(Data_ZSD10_Exito_Cencosub_ZVMI)
Data_ZSD10_Exito_Cencosub_ZVMI=Funtions.default_column(Columns_Data_ZSD10_Exito_Cencosub_ZVMI,Data_ZSD10_Exito_Cencosub_ZVMI)
Data_ZSD10_Exito_Cencosub_ZVMI=Data_ZSD10_Exito_Cencosub_ZVMI[Data_ZSD10_Exito_Cencosub_ZVMI["Cliente"].notnull()]

# Transformo datos 
Data_ZSD10_Exito_Cencosub_ZVMI["Unidades"],Data_ZSD10_Exito_Cencosub_ZVMI["ValorNeto"],Data_ZSD10_Exito_Cencosub_ZVMI["Total Pedidos"]=Funtions.Clean_Num_List(Data_ZSD10_Exito_Cencosub_ZVMI["Unidades"],Data_ZSD10_Exito_Cencosub_ZVMI["ValorNeto"],Data_ZSD10_Exito_Cencosub_ZVMI["Total Pedidos"])
# Tranformo la columna que tiene .0 por si es necesario
try:
    Data_ZSD10_Exito_Cencosub_ZVMI['Doc.venta']=Funtions.Clean_int_to_str(Data_ZSD10_Exito_Cencosub_ZVMI['Doc.venta'])
except:
    None


In [118]:
Datos_Agenda_Exito_ZVMI=list(Tabla_Agenda_Exito_ZVMI["Nº pedido cliente"].unique())    #Utilizo los valores de la tabla ya que con estos pedidos del Exito busco en la ZSD10
Datos_Agenda_Cencosub_ZVMI=list(Tabla_Agenda_Cencosub_ZVMI["Nº pedido cliente"].unique()) #Utilizo los valores de la tabla ya que con estos pedidos de Cencosub busco en la ZSD10

In [119]:
Data_ZSD10_Exito_ZVMI=Data_ZSD10_Exito_Cencosub_ZVMI.loc[Data_ZSD10_Exito_Cencosub_ZVMI["Nº pedido cliente"].isin(Datos_Agenda_Exito_ZVMI)]    
Data_ZSD10_Cencosub_ZVMI=Data_ZSD10_Exito_Cencosub_ZVMI.loc[Data_ZSD10_Exito_Cencosub_ZVMI["Nº pedido cliente"].isin(Datos_Agenda_Cencosub_ZVMI)]         #in list(Datos_Agenda_Exito_ZVMI)

In [120]:
#Resumenes numericos de tablas Cencosub

Resumen_Exito_ZVMI=Data_ZSD10_Exito_ZVMI.pivot_table(index=['Material'],aggfunc={'Nº pedido cliente':'count','ValorNeto':sum,'Unidades':sum,'Total Pedidos':sum}).reset_index()
Resumen_Cencosub_ZVMI=Data_ZSD10_Cencosub_ZVMI.pivot_table(index=['Material'],aggfunc={'Nº pedido cliente':'count','ValorNeto':sum,'Unidades':sum,'Total Pedidos':sum}).reset_index()

In [121]:
Lineas_Tabla_Agenda_Exito_ZVMI=Resumen_Exito_ZVMI['Nº pedido cliente'].sum()
Lineas_Tabla_Agenda_Cencosub_ZVMI=Resumen_Cencosub_ZVMI['Nº pedido cliente'].sum()
Unidades_Tabla_Agenda_Exito_ZVMI=Resumen_Exito_ZVMI.Unidades.sum()
Unidades_Tabla_Agenda_Cencosub_ZVMI=Resumen_Cencosub_ZVMI.Unidades.sum()
Valor_neto_Tabla_Agenda_Exito_ZVMI=Resumen_Exito_ZVMI['ValorNeto'].sum()
Valor_neto_Tabla_Agenda_Cencosub_ZVMI=Resumen_Cencosub_ZVMI['ValorNeto'].sum()
Total_Pedidos_Agenda_Exito_ZVMI=Resumen_Exito_ZVMI['Total Pedidos'].sum()
Total_Pedidos_Agenda_Cencosub_ZVMI=Resumen_Cencosub_ZVMI['Total Pedidos'].sum()

In [122]:
Lineas_Tabla_Agenda_Exito_ZVMI+Lineas_Tabla_Agenda_Cencosub_ZVMI

7994

In [123]:
Unidades_Tabla_Agenda_Exito_ZVMI+Unidades_Tabla_Agenda_Cencosub_ZVMI

18144.0

In [124]:
Valor_neto_Tabla_Agenda_Exito_ZVMI+Valor_neto_Tabla_Agenda_Cencosub_ZVMI

257338092.0

In [125]:
Total_Pedidos_Agenda_Exito_ZVMI+Total_Pedidos_Agenda_Cencosub_ZVMI

248.0

# Con los pedidos de la ZSD10 Busco a una nueva tansaacion los agotados ZSD038

In [126]:
# Utlizo dataframe de Documento de ventas de la ZSD10
Documento_Venta_Exito_Cencosub=Data_ZSD10_Exito_Cencosub_ZVMI['Doc.venta']


In [127]:
Documento_Venta_Exito_Cencosub

0       903118836
1       903118836
2       903118836
3       903118836
4       903118836
          ...    
7989    903122907
7990    903122907
7991    903122907
7992    903122758
7993    903122723
Name: Doc.venta, Length: 7994, dtype: object

In [128]:
def Search_Pedidos_ZSD038(Transsaccion,Series,session):      #(column Dataframe)
    """
    Transsaccion: Transsacion a buscar
    Series: Columna del dataframe que quiero copiar
    session: session del usuario
    provision: disposicion de interes
    """
    session.StartTransaction(Transsaccion)
    Series=Series.to_clipboard(index=False, header=False)
    session.findById("wnd[0]/usr/btn%_S_VBELN_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/tbar[1]/btn[8]").press()

Transsaccion="ZSD038"


Search_Pedidos_ZSD038(Transsaccion,Documento_Venta_Exito_Cencosub,session)

In [129]:
#Dowloader data

Name_ZSD038_Exito_Cencosub_Rechazados="Exito_Cencosub_Rechazados"
Ruta_Exito_Cencosub_ZSD010_Rechazados="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_ZSD038_Exito_Cencosub_Rechazados,session,Ruta_Exito_Cencosub_ZSD010_Rechazados)


In [130]:
url_ZSD038_Exito_Cencosub_Rechazados=Ruta_Exito_Cencosub_ZSD010_Rechazados+Name_ZSD038_Exito_Cencosub_Rechazados+".txt"
Data_ZSD038_Exito_Cencosub_Rechazados=pd.read_csv(url_ZSD038_Exito_Cencosub_Rechazados,delimiter="\t")
Data_ZSD038_Exito_Cencosub_Rechazados=Funtions.Clean_Columns(Data_ZSD038_Exito_Cencosub_Rechazados)
Data_ZSD038_Exito_Cencosub_Rechazados=Funtions.default_column(Columns_Data_ZSD038_Exito_Cencosub_Rechazados,Data_ZSD038_Exito_Cencosub_Rechazados)
Data_ZSD038_Exito_Cencosub_Rechazados=Data_ZSD038_Exito_Cencosub_Rechazados[Data_ZSD038_Exito_Cencosub_Rechazados["Doc.comer."].notnull()]
#Data_ZSD038_Exito_Cencosub_Rechazados=Data_ZSD10_Exito_Cencosub_ZVMI[Data_ZSD10_Exito_Cencosub_ZVMI["Cliente"].notnull()]
Data_ZSD038_Exito_Cencosub_Rechazados["Doc.comer."]=Data_ZSD038_Exito_Cencosub_Rechazados["Doc.comer."].astype(int)

In [131]:
Lista_Data_ZSD10_Exito_ZVMI=Data_ZSD10_Exito_ZVMI["Doc.venta"].unique()
Lista_Data_ZSD10_Cencosub_ZVMI=Data_ZSD10_Cencosub_ZVMI["Doc.venta"].unique()

In [132]:
Data_ZSD038_Exito_Cencosub_Rechazados["Doc.comer."]=Data_ZSD038_Exito_Cencosub_Rechazados["Doc.comer."].astype(int).astype(str)

In [133]:
Data_ZSD038_Exito_Rechazados=Data_ZSD038_Exito_Cencosub_Rechazados[Data_ZSD038_Exito_Cencosub_Rechazados["Doc.comer."].isin(Lista_Data_ZSD10_Exito_ZVMI)]
Data_ZSD038_Cencosub_Rechazados=Data_ZSD038_Exito_Cencosub_Rechazados[Data_ZSD038_Exito_Cencosub_Rechazados["Doc.comer."].isin(Lista_Data_ZSD10_Cencosub_ZVMI)]

In [134]:
['Doc.comer.', 'Fecha doc.', 'Destinat.', 'Nombre 1', 'Cód.Vend.',
       'Nombre vendedor', 'MR', 'Denominación', 'Material', 'Denominación.1',
       'Marca', 'Categoría', 'Ctd.confirmada', 'Valor neto', 'Mon.', 'CDis',
       'Solic.', 'Nombre']

['Doc.comer.',
 'Fecha doc.',
 'Destinat.',
 'Nombre 1',
 'Cód.Vend.',
 'Nombre vendedor',
 'MR',
 'Denominación',
 'Material',
 'Denominación.1',
 'Marca',
 'Categoría',
 'Ctd.confirmada',
 'Valor neto',
 'Mon.',
 'CDis',
 'Solic.',
 'Nombre']

In [135]:
def Search_Pedidos_ZSD037(Transsaccion,Series,session,provision=None):      #(column Dataframe)
    """
    Transsaccion: Transsacion a buscar
    Series: Columna del dataframe que quiero copiar
    session: session del usuario
    provision: disposicion de interes
    """
    session.StartTransaction(Transsaccion)
    Series=Series.to_clipboard(index=False, header=False)
    session.findById("wnd[0]/usr/btn%_SP$00011_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    if provision!=None:
        session.findById("wnd[0]/usr/ctxt%LAYOUT").text = provision
    session.findById("wnd[0]/tbar[1]/btn[8]").press()


In [27]:
x="      asdasdasdas      "

import re
re.findall("(?=\s*\w*\s*",x)

['      ', 'asdasdasdas      ', '']